## Classification: Instant Recognition With Caffe
In this example we'll classify an image with the bundled CaffeNet model (which is based on the network architecture of Krizhevsky et al. for ImageNet).

We'll compare CPU and GPU modes and then dig into the model to inspect features and the output.

### 1. Setup
- First, set up Python, numpy and matplotlib

In [1]:
# set up Python Enviornment: numpy for numerical routines and matplotlin for plotting

import numpy as np 
import matplotlib.pyplot as plt 
# display plot in this notebook
%matplotlib inline 

#set display defaults 
# large images 
plt.rcParams['figure.figsize'] = (10,10) 
# don't  interplolate: show square pixels 
plt.rcParams['image.interpolation'] = 'nearest' 
# use grayscale output rather than a (potenstially misleading) color heatmap
plt.rcParams['image.cmap'] = 'gray'


- Load Caffe

In [2]:
# The caffe module needs to be on the Python path;
# we'll add it here explicitly 
import sys 
# this file should be run from {caffe_root}/examples (otherwise change this line)
caffe_root = '/home/amanullahtariq/caffe/'
sys.path.insert(0,caffe_root + 'python')

# If you get "No module named _caffe", either you have not built pycaffe or you have the wrong path.
import caffe

ImportError: /home/amanullahtariq/anaconda2/lib/python2.7/site-packages/zmq/backend/cython/../../../../.././libstdc++.so.6: version `GLIBCXX_3.4.21' not found (required by /home/amanullahtariq/caffe/python/caffe/_caffe.so)

- If needed, download the reference model ("CaffeNet", a variant of AlexNet).

In [15]:
import os
if os.path.isfile(caffe_root + 'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel'):
    print 'CaffeNet found.'
else:
    print 'Downloading pre-trained CaffeNet model...'
    !/home/amanullahtariq/caffe/scripts/download_model_binary.py /home/amanullahtariq/caffe/models/bvlc_reference_caffenet

CaffeNet found.


### 2. Load net and set up input preprocessing
- Set Caffe to CPU mode and load the net from disk

In [18]:
caffe.set_mode_cpu()

model_def = caffe_root + 'models/bvlc_reference_caffenet/deploy.prototxt'
model_weights = caffe_root + 'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel'

net = caffe.Net(model_def, # defines the structure of the model
               model_weights, # contains the trained weights
               caffe.TEST) # use test mode (e.g. don't perform dropout)

- Set up input preprocessing. (We'll use Caffe's caffe.io. Transformer to do this, but this step is independent of other parts of caffe, so any custom preprocessing code may be used).

- Our default CaffeNet is configured to take images in BGR format. Values are expected to start in the range[0, 255] and then have the mean ImageNet pixel value subtracted from them. In addition, the channel dimension is expected as the first (outermost) dimension. 

- As matlplotlib will load images with the values in the range [0,1] in RGB format with the channel as the innermost dimension, we are arranging for the need transformations here. 

In [36]:
# load the mean ImageNet image (as distributed with Caffe) for subtraction
mu = np.load(caffe_root + 'python/caffe/imagenet/ilsvrc_2012_mean.npy')
mu = mu.mean(1).mean(1) # average over pixels to obtain the mean BGR pixel values 
print ('mean-subtracted values: ', zip('BGR', mu))

# create transformer for the input called 'data'
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})

# move image channel to outermost dimension
transformer.set_transpose('data', (2,0,1))
# subtract the dataset-mean value in each element
transformer.set_mean('data',mu)
# rescale from [0,1] to [0, 255]
transformer.set_raw_scale('data',255)
#swap channels from RGB to BGR
transformer.set_channel_swap('data', (2,1,0))

('mean-subtracted values: ', [('B', 104.0069879317889), ('G', 116.66876761696767), ('R', 122.6789143406786)])


### 3. CPU Classification
- Now we're ready to perform classification. 


## Reference
- Classification: Instant Recognition with Caffe (http://nbviewer.jupyter.org/github/BVLC/caffe/blob/master/examples/00-classification.ipynb)